In [45]:
import networkx as nx
import os
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
import csv
import copy
from scipy.optimize import curve_fit
import collections

In [2]:
# Reading in the union network saved after data prep
ports = ['1', '2']
years = ['2005', '2010', '2015', '2017']
codes = ['2701', '2709', '2711']

GUnion = {(year, code): nx.DiGraph() for year in years for code in codes}

for gKey in GUnion:
    GUnion[gKey] = nx.read_gpickle("PycharmProjects/EnergyTradeAnalysis/Database/GPickleData/GUnion_" + str(gKey) + ".gp")

In [3]:
# Creating a dict with sum of node weights and assigning it to strength variable for each node
for gKey in GUnion:
    nodeAttrs = {}
    for node in GUnion[gKey].nodes():
        weightNode = 0
        for edge in GUnion[gKey].edges():
            if node == edge[0] or node == edge[1]:
                weightNode += GUnion[gKey][edge[0]][edge[1]]['weight']
        nodeAttrs[node] = {'strength': weightNode}
    nx.set_node_attributes(GUnion[gKey], nodeAttrs)

In [4]:
# Getting nodeStrength in dictionary for all 12 networks
nodeStrength = {}
for gKey in GUnion:
    nodeStrength[gKey] = pd.DataFrame.from_dict(dict(GUnion[gKey].nodes('strength')), orient='index')
    nodeStrength[gKey].sort_values(by=0, ascending=False).to_csv('PycharmProjects/EnergyTradeAnalysis/Results/NodeStrengthUnion/NodeStrUnion_' + str(gKey) + '.csv', mode = 'w')

In [38]:
# Node Strength Log Histogram

for year in years:
    for code in codes:    
        x = list(nodeStrength[year, code][0])
        x = pd.Series(x)

        plt.figure(figsize=(10, 7.5))

        ax = plt.subplot(111)
        ax.get_xaxis().tick_bottom()  
        ax.get_yaxis().tick_left()  

        plt.xticks(fontsize=14)  
        plt.yticks(fontsize=14)  

        plt.xlabel("Log Node Strength", fontsize=16)  
        plt.ylabel("Frequency", fontsize=16)  

        # histogram on log scale. 
        # Use non-equal bin sizes, such that they look equal on log scale.
        #logbins = np.logspace(np.log10(x.min()),np.log10(x.max()),50)
        logbins = np.geomspace(x.min()+0.000001, x.max(), 50)
        plt.hist(x, color="#3F5D7D", ec='black', bins=logbins)
        plt.xscale('log')
        plt.savefig("PycharmProjects/EnergyTradeAnalysis/Results/GraphsUnion/NodeStrength/NodeStr_" + str(code) + "_" + str(year) + ".png", bbox_inches='tight', format='png', dpi=300)
        plt.clf()


<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

...

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

In [39]:
centrality = {}
in_centrality = {}
out_centrality = {}
eigen_centrality = {}
in_degree = {}
out_degree = {}
degree = {}

for graphKey in GUnion:
    centrality[graphKey] = nx.degree_centrality(GUnion[graphKey])
    in_centrality[graphKey] = nx.in_degree_centrality(GUnion[graphKey])
    out_centrality[graphKey] = nx.out_degree_centrality(GUnion[graphKey])
    eigen_centrality[graphKey] = nx.eigenvector_centrality_numpy(GUnion[graphKey], weight = 'weight')
    in_degree[graphKey] = {x: GUnion[graphKey].in_degree(x) for x in GUnion[graphKey].nodes()}
    out_degree[graphKey] = {x: GUnion[graphKey].out_degree(x) for x in GUnion[graphKey].nodes()}
    degree[graphKey] = {x: GUnion[graphKey].degree(x) for x in GUnion[graphKey].nodes()}
        
results = {}
for year in years:
    for code in codes:
        results[year, code] = pd.DataFrame({
                                    'Centrality': list(centrality[year, code].values()), 
                                    'In-Centrality': list(in_centrality[year, code].values()), 
                                    'Out-Centrality': list(out_centrality[year, code].values()),
                                    'Eigen Centrality': list(eigen_centrality[year, code].values()),
                                    'Degree': list(degree[year, code].values()),
                                    'In-Degree': list(in_degree[year, code].values()), 
                                    'Out-Degree': list(out_degree[year, code].values())
                                     }, index = GUnion[year, code].nodes())


In [40]:
for year in years:
    for code in codes:
        results[year, code].sort_values(by=['Degree'], ascending=False).to_csv("PycharmProjects/EnergyTradeAnalysis/Results/DetailsUnion/Details_" + str(year) + "_" + str(code) + ".csv")               

In [78]:
# Exclude couple of nodes which have 0 degree
for year in years:
    for code in codes:
        results[year, code] =  results[year, code][results[year, code].Degree != 0]

In [54]:
# Power Law Fitting in next few cells
# Function to calculate the power-law with constants a and b
def power_law(x, a, b):
    return a*np.power(x, b)

In [79]:
paramType = 'Degree'
params = {}

for year in years:
    for code in codes[:3]:
        dataList = list(results[year, code][paramType])
        
        # Getting Frequency and Degree in separate but similar lists
        # Degree - x values, Frequency - y values
        counter=collections.Counter(dataList)
        df = pd.DataFrame(columns=['Degree','Frequency'])
        for k, v in counter.items():
            df.loc[len(df)] = [k, v]
        df = df.sort_values(by = 'Degree')

        fig, ax = plt.subplots()

        # Set the x and y-axis scaling to logarithmic
        ax.set_xscale('log')
        ax.set_yscale('log')

        # Edit the major and minor tick locations of x and y axes
        ax.set_xlim(0.5, 300)
        ax.set_ylim(0.5, 200)
        plt.xticks(fontsize=14)  
        plt.yticks(fontsize=14)  

        ax.set_ylabel('Frequency', fontsize=16)
        ax.set_xlabel('Degree',  fontsize=16)
        ax.scatter(list(df['Degree']), list(df['Frequency']), s = 10, color="#3F5D7D", label='data')

        # Fit the dummy power-law data
        pars, cov = curve_fit(f=power_law, xdata=list(df['Degree']), ydata=list(df['Frequency']), p0=[0, 0], bounds=(-np.inf, np.inf))

        params[year, code] = pars

        ax.plot(list(df['Degree']), power_law(list(df['Degree']), *pars), color="#FF7F0E", label='fit: a=%5.3f, b=%5.3f' % tuple(pars))
        ax.legend(prop={'size': 14})
        
        figname = 'PycharmProjects/EnergyTradeAnalysis/Results/Fit/DegDistrFit' + '_' + code + "_" + year 
        plt.savefig(figname +'.png', bbox_inches='tight', dpi=300)
        plt.clf()


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [82]:
params
with open('PycharmProjects/EnergyTradeAnalysis/Results/Fit/degDistrPowerLawParams.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["year","code", "a", "b"])
    for k,v in params.items():
        writer.writerow([k[0], k[1], v[0], v[1]])

In [86]:
# Eigenvector Centrality Histogram

for year in years:
    for code in codes:    
        x = list(results[year, code]["Eigen Centrality"])
        x = pd.Series(x)
        
        plt.figure(figsize=(10, 7.5))

        ax = plt.subplot(111)
        ax.get_xaxis().tick_bottom()  
        ax.get_yaxis().tick_left()  

        plt.xticks(fontsize=14)  
        plt.yticks(fontsize=14)  

        plt.xlabel("Eigen Centrality", fontsize=16)  
        plt.ylabel("Frequency", fontsize=16)  
    
        bins = np.linspace(x.min(), x.max(), 100)
        plt.hist(x, color="#3F5D7D", ec='black', bins=bins)
        plt.savefig("PycharmProjects/EnergyTradeAnalysis/Results/GraphsUnion/Centrality/EigenCentr_" + str(code) + "_" + str(year) + ".png", bbox_inches='tight', format='png', dpi=300)
        plt.clf()


<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

...

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>

<Figure size 720x540 with 0 Axes>